# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/data-engineering/01_data_modeling/Project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['cassandra-seed'], port=9042) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
data = pd.read_csv('event_datafile_new.csv')

In [10]:
data.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

In [28]:
data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [45]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [63]:
# query = "DROP TABLE IF EXISTS music_app_history "
# try:
#     session.execute(query)
# except Exception as e:
#     print(e)

In [20]:
# file = 'event_datafile_new.csv'

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#         print(len(line))
#         break

11


In [50]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into music_app_history (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),line[6],line[7], int(line[8]),line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [57]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select count(*) from music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(count=6820)


In [59]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "select artist, song, length from music_app_history where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [67]:
query = "DROP TABLE IF EXISTS music_app_history "
try:
    session.execute(query)
except Exception as e:
    print(e)

In [64]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int, PRIMARY KEY (sessionId, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [65]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into music_app_history (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),line[6],line[7], int(line[8]),line[9], int(line[10])))

In [66]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select artist, song, firstName, lastName from music_app_history where sessionId=182 and userid=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)   

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [77]:
query = "DROP TABLE IF EXISTS music_app_history "
try:
    session.execute(query)
except Exception as e:
    print(e)

In [78]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int, PRIMARY KEY (song, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [79]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into music_app_history (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),line[6],line[7], int(line[8]),line[9], int(line[10])))

In [80]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select artist, song, firstName, lastName from music_app_history where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)   

                    

Row(artist='The Black Keys', song='All Hands Against His Own', firstname='Sara', lastname='Johnson')
Row(artist='The Black Keys', song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(artist='The Black Keys', song='All Hands Against His Own', firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [81]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS music_app_history "
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [82]:
session.shutdown()
cluster.shutdown()